In [1]:
import os
import sys
# Add path to notebook
sys.path.insert(0, '../')
from ctrace.constraint import load_graph, find_contours
from ctrace.contact_tracing import PQ
from ctrace.contact_tracing import *
from ctrace.constraint import ProbMinExposed, ProbMinExposedMIP
from ctrace.draw import *
os.getcwd()


'/home/ubuntu/ContactTracing'

In [2]:
LOADING = True
np.random.seed(42)

In [5]:
# Setup graph and infected sets
if LOADING:
    G = load_graph("montgomery", in_dir="data/mont/labelled")
    # Randomly sample 5% of infected
    N = G.number_of_nodes()
    num_infected = int(N * 0.05)
    I = np.random.choice(N, num_infected, replace=False)

    # Set K value
    K = int(0.8 * len(I))
else:
    G = nx.balanced_tree(3, 3)
    I = {0, 1}

    # Set K value
    K = 3

# Find contours
contour1, contour2 = find_contours(G, I)

# Find the infected probabilities
p1, q = PQ_deterministic(G, I, contour1, 0.1)


FileNotFoundError: [Errno 2] No such file or directory: '../data/mont/labelled/montgomery/data.txt'

In [11]:
from ctrace.simulation import initial

SIR_init = initial(G=G, timesteps=5, p=0.1, cache="test.json")
print(len(SIR_init[1]))

[  8  14  44 117 273 564]
564


In [13]:
SIR = initial(G, from_cache="test.json")
print(len(SIR[1]))

564


In [14]:
print(K)
print(p1, q)

3017
{0: 0.9, 1: 0.81, 2: 0.81, 3: 0.9, 7: 0.9, 12: 0.7290000000000001, 13: 0.9, 15: 0.81, 17: 0.81, 18: 0.9, 19: 0.9, 20: 0.9, 22: 0.9, 23: 0.81, 24: 0.81, 30: 0.81, 31: 0.9, 33: 0.9, 35: 0.7290000000000001, 36: 0.5904900000000001, 38: 0.9, 40: 0.81, 41: 0.7290000000000001, 44: 0.9, 45: 0.9, 49: 0.9, 51: 0.9, 52: 0.9, 54: 0.9, 61: 0.9, 64: 0.7290000000000001, 65: 0.9, 66: 0.7290000000000001, 68: 0.81, 69: 0.9, 71: 0.9, 72: 0.9, 73: 0.9, 75: 0.9, 78: 0.9, 79: 0.7290000000000001, 82: 0.9, 83: 0.9, 86: 0.4782969000000001, 87: 0.531441, 89: 0.9, 90: 0.9, 92: 0.9, 94: 0.81, 95: 0.81, 96: 0.9, 97: 0.7290000000000001, 98: 0.9, 99: 0.6561, 100: 0.81, 104: 0.9, 106: 0.9, 108: 0.6561, 112: 0.81, 113: 0.9, 117: 0.9, 118: 0.81, 119: 0.9, 120: 0.81, 121: 0.9, 123: 0.9, 124: 0.9, 125: 0.9, 126: 0.9, 128: 0.9, 131: 0.81, 132: 0.7290000000000001, 133: 0.81, 140: 0.9, 142: 0.9, 143: 0.9, 144: 0.9, 146: 0.81, 147: 0.9, 148: 0.81, 149: 0.9, 152: 0.9, 155: 0.9, 156: 0.9, 157: 0.7290000000000001, 166: 0.9

In [ ]:
if not LOADING:
    draw_contours(G, I, contour1, contour2)

In [ ]:
print(f"K: {K}")
print(f"I: {I}")

In [ ]:
constraintMIP = ProbMinExposedMIP.from_dataframe(G, I, contour1, contour2, p1, q, K)
constraintNone = ProbMinExposedMIP.from_dataframe(G, I, contour1, contour2, p1, q, 0)

In [ ]:
constraintMIP.solve_lp()
constraintNone.solve_lp()

In [ ]:
constraintMIP.getVariables()

In [ ]:
quarantined = constraintMIP.quarantined_solution
saved = constraintMIP.saved_solution

print(quarantined)
print(saved)

In [ ]:
# Solution Visualization
draw_prob(G, I, contour1, contour2, quarantined, saved, constraintMIP.p1, constraintMIP.q)

In [ ]:
draw_prob(G, I,
          contour1,
          contour2,
          constraintNone.quarantined_solution,
          constraintNone.saved_solution,
          constraintNone.p1,
          constraintNone.q)

In [ ]:
constraintNone.saved_solution

In [ ]:
saved

In [ ]:
[-np.log(1 - x) for _, x in saved.items()]